In [30]:
# Set up virtual environment
## PS> cd <repo>
## PS> python -m venv venv
## PS> venv\Scripts\activate #  for linux/gitbash: source venv/Scripts/activate

# Load required packages
## (venv) $ python -m pip install dash==2.8.1 pandas==1.5.3

# streamlit run streamlit_app.py

In [59]:
# %pip freeze > requirements2.txt
# %pip install -r requirements.txt


In [1]:
%load_ext autoreload
%autoreload 2

import sys
sys.path.insert(1, '../')

# Run individual scripts

## Set up

In [131]:
from utils_logging import close_log_handlers
close_log_handlers()

In [3]:
import logging

from utils_logging import setup_logging
from utils import read_yaml, absolute_path, create_directory

# Setup logging
close_log_handlers()
setup_logging()
logger = logging.getLogger(__name__)
logger.info("Start of Streamlit App.")

# Setup folder locations
dataset_yamls_folder = absolute_path('dataset_yamls')
data_folder = absolute_path('data')
create_directory(data_folder)
outputs_folder = absolute_path('outputs')
create_directory(outputs_folder)

# Select dataset

In [4]:
# Read yaml for options to display
dataset_options_yaml = read_yaml('dataset_options.yaml')

# Select what data to look at
options_list = list(dataset_options_yaml.keys())
print(f"Options list: {options_list}")
selected_dataset_yaml_list = options_list[0]
print(f'selected_dataset_yaml_list:{selected_dataset_yaml_list}')

Options list: ['Interest Rates and Yields – Money Market – Daily – F1']
selected_dataset_yaml_list:Interest Rates and Yields – Money Market – Daily – F1


## Load data

In [5]:
from data_loading import data_loader
# Download the required data
data_dict = data_loader(
    dataset_yamls_folder = dataset_yamls_folder,
    dataset_yaml_name_list = dataset_options_yaml[selected_dataset_yaml_list],
    data_folder = data_folder,
)

In [9]:
print(data_dict.keys())
data_dict['rba_f01d'].keys()


dict_keys(['rba_f01d'])


dict_keys(['file_path', 'header_descriptions_df', 'cleaned_df'])

In [132]:
data_dict['rba_f01d']['cleaned_df']

,Date,Cash Rate Target,Change in the Cash Rate Target,Interbank Overnight Cash Rate,Highest Interbank Overnight Cash Rate,Lowest Interbank Overnight Cash Rate,Volume of Cash Market Transactions,Number of Cash Market Transactions,Total Return Index,EOD 1-month BABs/NCDs,EOD 3-month BABs/NCDs,EOD 6-month BABs/NCDs,1-month OIS,3-month OIS,6-month OIS,1-month Treasury Note,3- month Treasury Note,6- month Treasury Note
0,2011-01-05,0.0475,NaN,0.0475,NaN,NaN,NaN,NaN,1.000130,0.048300,0.049850,0.051400,0.047600,0.047875,0.048500,0.04805,0.04835,NaN
1,2011-01-06,0.0475,NaN,0.0475,NaN,NaN,NaN,NaN,1.000260,0.048300,0.049800,0.051500,0.047600,0.047865,0.048500,0.04800,0.04830,NaN
2,2011-01-07,0.0475,NaN,0.0475,NaN,NaN,NaN,NaN,1.000390,0.048300,0.050100,0.051500,0.047600,0.047900,0.048600,0.04805,0.04830,NaN
3,2011-01-10,0.0475,NaN,0.0475,NaN,NaN,NaN,NaN,1.000781,0.048300,0.050000,0.051450,0.047565,0.047840,0.048465,0.04805,NaN,NaN
4,2011-01-11,0.0475,NaN,0.0475,NaN,NaN,NaN,NaN,1.000911,0.048200,0.049400,0.050750,0.047500,0.047700,0.048000,0.04790,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3320,2024-02-19,0.0435,NaN,0.0432,0.0432,0.0432,3.460000e+08,6.0,1.317118,0.042975,0.043400,0.044563,NaN,NaN,NaN,NaN,NaN,NaN
3321,2024-02-20,0.0435,NaN,0.0432,0.0435,0.0432,1.581000e+09,12.0,1.317274,0.042975,0.043329,0.044600,NaN,NaN,NaN,NaN,NaN,NaN
3322,2024-02-21,0.0435,NaN,0.0432,0.0432,0.0432,1.605000e+09,10.0,1.317430,0.042944,0.043326,0.044500,NaN,NaN,NaN,NaN,NaN,NaN
3323,2024-02-22,0.0435,NaN,0.0432,0.0432,0.0432,1.060000e+09,7.0,1.317586,0.042856,0.043273,0.044510,NaN,NaN,NaN,NaN,NaN,NaN


## Charting

In [11]:
import plotly.express as px
import pandas as pd

In [12]:

def plotly_line_chart(df, date_column, value_columns, colors):
    """
    Generates a Plotly line chart.

    :param df: DataFrame containing the data.
    :param date_column: The name of the column in df that contains date values.
    :param value_columns: A list of column names to be plotted.
    :param colors: A dictionary mapping column names to colors.
    """
    # Initialize a figure
    fig = px.line()

    # Check if colors for all value columns are provided, otherwise use default colors
    for column in value_columns:
        if column in colors:
            color = colors[column]
        else:
            color = None

        fig.add_scatter(x=df[date_column], y=df[column], mode='lines', name=column, line=dict(color=color))

    # Update layout
    fig.update_layout(title='Line Chart',
                      xaxis_title=date_column,
                      yaxis_title='Values',
                      legend_title='Variables')

    fig.show()


In [38]:


# Columns to plot and their colors
# value_columns = ['Value1', 'Value2']
# colors = {'Value1': 'blue', 'Value2': 'red'}

# Generate chart
plotly_line_chart(
    df = data_dict['rba_f01d']['cleaned_df'],
    date_column = 'Date',
    value_columns = ['Cash Rate Target'],
    colors = {'Cash Rate Target': '#D62728', 'Cash Rate Target2': 'red'},
)

# data_dict['rba_f01d']['cleaned_df']['Cash Rate Target']
# 
# , value_columns, colors
# 

In [40]:
# data_dict['rba_f01d']['cleaned_df'].columns


colors_dict = {
    'Cash Rate Target': '#D62728',
    'Change in the Cash Rate Target ': '#1F77B4',
    'Interbank Overnight Cash Rate': '#2CA02C',
    'Highest Interbank Overnight Cash Rate': '#FF7F0E',
    'Lowest Interbank Overnight Cash Rate': '#9467BD',
    'Volume of Cash Market Transactions': '#8C564B',
    'Number of Cash Market Transactions': '#E377C2',
    'Total Return Index': '#7F7F7F',
    'EOD 1-month BABs/NCDs': '#17BECF',
    'EOD 3-month BABs/NCDs': '#BCBD22',
    'EOD 6-month BABs/NCDs': '#329F9D',
    '1-month OIS': '#C71585',
    '3-month OIS': '#000080',
    '6-month OIS': '#FF6347',
    '1-month Treasury Note': '#FFD700',
    '3- month Treasury Note': '#E6E6FA',
}

colors_dict = {
    'Cash Rate Target': '#1F77B4',
    'Change in the Cash Rate Target ': '#D62728',
    'Interbank Overnight Cash Rate': '#2CA02C',
    'Highest Interbank Overnight Cash Rate': '#FFD700',
    'Lowest Interbank Overnight Cash Rate': '#7F7F7F',
    # 'Volume of Cash Market Transactions': ,
    # 'Number of Cash Market Transactions': ,
    'Total Return Index': '#4682B4',
    # 'EOD 1-month BABs/NCDs': ,
    'EOD 3-month BABs/NCDs': '#FF7F0E',
    'EOD 6-month BABs/NCDs': '#9467BD',
    # '1-month OIS': ,
    # '3-month OIS': ,
    # '6-month OIS': ,
    # '1-month Treasury Note': ,
    # '3- month Treasury Note': ,
}

#1F77B4
#D62728
#2CA02C
#FFD700
#7F7F7F
#FF7F0E
#9467BD
#4682B4

# Dark Blue: #1F77B4 - Conveys trust and stability.
# Crimson Red: #D62728 - Good for highlighting declines or critical data points.
# Forest Green: #2CA02C - Often associated with growth and positive movement.
# Gold: #FFD700 - Useful for highlighting premium or significant data series.
# Slate Gray: #7F7F7F - Neutral, good for background or less significant data series.
# Burnt Orange: #FF7F0E - Stands out well against both light and dark backgrounds.
# Plum: #9467BD - Offers a distinct but conservative option for diversifying the palette.
# Steel Blue: #4682B4 - Less intense than dark blue, useful for differentiation without overwhelming.


In [42]:
# Initialize a figure
fig = px.line()

# Check if colors for all value columns are provided, otherwise use default colors
for column in value_columns:
    if column in colors:
        color = colors[column]
    else:
        color = None

    fig.add_scatter(x=df[date_column], y=df[column], mode='lines', name=column, line=dict(color=color))

# Update layout
fig.update_layout(title='Line Chart',
                    xaxis_title=date_column,
                    yaxis_title='Values',
                    legend_title='Variables')

['Cash Rate Target',
 'Change in the Cash Rate Target ',
 'Interbank Overnight Cash Rate',
 'Highest Interbank Overnight Cash Rate',
 'Lowest Interbank Overnight Cash Rate',
 'Total Return Index',
 'EOD 3-month BABs/NCDs',
 'EOD 6-month BABs/NCDs']

In [44]:
cols = data_dict['rba_f01d']['cleaned_df'].columns
cols = cols[cols != 'Date']
cols = cols[cols != 'Volume of Cash Market Transactions']
cols = cols[cols != 'Number of Cash Market Transactions']
cols


colors_dict = {
    'Cash Rate Target': '#1F77B4',
    'Change in the Cash Rate Target ': '#D62728',
    'Interbank Overnight Cash Rate': '#2CA02C',
    'Highest Interbank Overnight Cash Rate': '#FFD700',
    'Lowest Interbank Overnight Cash Rate': '#7F7F7F',
    # 'Volume of Cash Market Transactions': ,
    # 'Number of Cash Market Transactions': ,
    # 'Total Return Index': ,
    'EOD 1-month BABs/NCDs': '#4682B4',
    'EOD 3-month BABs/NCDs': '#FF7F0E',
    'EOD 6-month BABs/NCDs': '#9467BD',
    # '1-month OIS': ,
    # '3-month OIS': ,
    # '6-month OIS': ,
    # '1-month Treasury Note': ,
    # '3- month Treasury Note': ,
}


In [129]:
tf_line_chart_dict = {
    'title': 'Annualised Interest Rates for BABs/NCDs',
    'dale_column': 'Date',
    'lines_dict': {
        'EOD 1-month BABs/NCDs': '#43B7C2',
        'EOD 3-month BABs/NCDs': '#024B79',
        'EOD 6-month BABs/NCDs': '#FFAD48',
    },
}
tf_line_chart_dict

# Title
tf_chart_design_dict={
    'title_dict': {
        # Title should already be defined
        'y':0.95,
        'x':0.5,
        'xanchor': 'center',
        'yanchor': 'top',
        'font': {
            'color': '#1E3A2F',
            'size': 40,
            'family': "Avenir Next LT Pro Demi",
        }
    },
}

In [130]:
from charting.utils_charting import tf_line_chart
tf_line_chart(
    data_frame = data_dict['rba_f01d']['cleaned_df'],
    x = tf_line_chart_dict['dale_column'],
    y = list(tf_line_chart_dict['lines_dict'].keys()),
    color_discrete_map=tf_line_chart_dict['lines_dict'],
    title=tf_line_chart_dict['title'],
    height=800,
    tf_title_dict=tf_chart_design_dict['title_dict'],
)

{'title': 'Annualised Interest Rates for BABs/NCDs',
 'dale_column': 'Date',
 'lines_dict': {'EOD 1-month BABs/NCDs': '#43B7C2',
  'EOD 3-month BABs/NCDs': '#024B79',
  'EOD 6-month BABs/NCDs': '#FFAD48'}}

In [100]:
fig = px.line(
    data_frame=data_dict['rba_f01d']['cleaned_df'],
    x=line_chart_dict['dale_column'],
    y=list(line_chart_dict['lines_dict'].keys()),
    line_group=None,
    color=None,
    line_dash=None,
    symbol=None,
    hover_name=None,
    hover_data=None,
    custom_data=None,
    text=None,
    facet_row=None,
    facet_col=None,
    facet_col_wrap=0,
    facet_row_spacing=None,
    facet_col_spacing=None,
    error_x=None,
    error_x_minus=None,
    error_y=None,
    error_y_minus=None,
    animation_frame=None,
    animation_group=None,
    category_orders=None,
    labels=None,
    orientation=None,
    color_discrete_sequence=None,
    color_discrete_map=line_chart_dict['lines_dict'],
    line_dash_sequence=None,
    line_dash_map=None,
    symbol_sequence=None,
    symbol_map=None,
    markers=False,
    log_x=False,
    log_y=False,
    range_x=None,
    range_y=None,
    line_shape=None,
    render_mode='auto',
    title=line_chart_dict['title'],
    template=None,
    width=None,
    height=800,
)

fig.update_xaxes(
    # tickformat=".0%",
    showgrid=True,
    gridwidth=1,
    gridcolor='grey',
)

fig.update_layout(

)

fig.update_layout(
    # Background
    plot_bgcolor='#FFFFFF',
    paper_bgcolor='#FFFFFF',

    # Y axis
    yaxis_title='Interest Rate',
    yaxis_tickformat='.2%',  # Formats ticks as percentages with 2 decimal places

    # Legend
    legend=dict(
        title='Legend',
        orientation="h",
        yanchor="bottom",
        y=-0.2,
        xanchor="right",
        x=1
    ),
)


In [58]:
colors_dict

plotly_line_chart(
    df = data_dict['rba_f01d']['cleaned_df'],
    date_column = line_chart_dict['dale_column'],
    # value_columns = list(line_chart_dict['lines_dict'].keys()),
    color = list(line_chart_dict['lines_dict'].keys()),
    # colors = list(line_chart_dict['lines_dict'].keys()),
)

TypeError: plotly_line_chart() got an unexpected keyword argument 'color'

In [ ]:
# Example usage
if __name__ == "__main__":
    # Sample DataFrame
    dates = pd.date_range(start='2021-01-01', end='2021-01-10')
    data = {
        'Date': dates,
        'Value1': range(10),
        'Value2': range(10, 20),
    }
    df = pd.DataFrame(data)

    # Columns to plot and their colors
    value_columns = ['Value1', 'Value2']
    colors = {'Value1': 'blue', 'Value2': 'red'}

    # Generate chart
    plotly_line_chart(df, 'Date', value_columns, colors)

In [182]:
from utils import absolute_path, create_directory

# Setup folder locations
yamls_folder = absolute_path('dataset_yamls')
data_folder = absolute_path('data')
create_directory(data_folder)
outputs_folder = absolute_path('outputs')
create_directory(outputs_folder)

In [180]:
yamls_folder

'C:\\Users\\tyewf\\github_projects\\rba_statistic_tables\\source_yamls'

## Text Generator

In [175]:
# Arguments
rba_f01d = data_dict['rba_f01d']['cleaned_df']


In [183]:
rba_f01d_cleaned = rba_f01d[~pd.isna(rba_f01d['Cash Rate Target'])].copy()

latest_date = pd.to_datetime(rba_f01d_cleaned['Date'].iloc[-1])

text = f"The Cash Rate Target as at {latest_date.strftime('%d %B %Y')} sits at {'{:.2%}'.format(rba_f01d_cleaned['Cash Rate Target'].iloc[-1])}."
print(text)

The Cash Rate Target as at 22 February 2024 sits at 4.35%.


In [185]:
rba_f01d_cleaned_12mths = rba_f01d_cleaned[rba_f01d_cleaned['Date'] >= (latest_date - pd.DateOffset(years=1))]
rba_f01d_cleaned_12mths['Change in the Cash Rate Target '].dropna()


3080    0.0025
3117    0.0025
3142    0.0025
3251    0.0025
Name: Change in the Cash Rate Target , dtype: float64

In [176]:
rba_f01d['Cash Rate Target']

.dropna(subset=target_columns_list).copy()

rba_f01d['Change in the Cash Rate Target']

0      NaN
1      NaN
2      NaN
3      NaN
4      NaN
        ..
3320   NaN
3321   NaN
3322   NaN
3323   NaN
3324   NaN
Name: Change in the Cash Rate Target , Length: 3325, dtype: float64

In [171]:
# Function
text_dict = {}

# Only select require columns and drop na values
df_clean = df[[date_column] + target_columns_list].dropna(subset=target_columns_list).copy()

for col in target_columns_list:
    print(df_clean[col])
    text = f"The {col} as at {df_clean[date_column].iloc[-1].strftime('%d %B %Y')} sits at {'{:.2%}'.format(df_clean[col].iloc[-1])}."
    print(text)
    
    



0       0.0475
1       0.0475
2       0.0475
3       0.0475
4       0.0475
         ...  
3319    0.0435
3320    0.0435
3321    0.0435
3322    0.0435
3323    0.0435
Name: Cash Rate Target, Length: 3324, dtype: float64
The Cash Rate Target as at 22 February 2024 sits at 4.35%


In [174]:
for val in df_clean[col]:
    print(val)

0.0475
0.0475
0.0475
0.0475
0.0475
0.0475
0.0475
0.0475
0.0475
0.0475
0.0475
0.0475
0.0475
0.0475
0.0475
0.0475
0.0475
0.0475
0.0475
0.0475
0.0475
0.0475
0.0475
0.0475
0.0475
0.0475
0.0475
0.0475
0.0475
0.0475
0.0475
0.0475
0.0475
0.0475
0.0475
0.0475
0.0475
0.0475
0.0475
0.0475
0.0475
0.0475
0.0475
0.0475
0.0475
0.0475
0.0475
0.0475
0.0475
0.0475
0.0475
0.0475
0.0475
0.0475
0.0475
0.0475
0.0475
0.0475
0.0475
0.0475
0.0475
0.0475
0.0475
0.0475
0.0475
0.0475
0.0475
0.0475
0.0475
0.0475
0.0475
0.0475
0.0475
0.0475
0.0475
0.0475
0.0475
0.0475
0.0475
0.0475
0.0475
0.0475
0.0475
0.0475
0.0475
0.0475
0.0475
0.0475
0.0475
0.0475
0.0475
0.0475
0.0475
0.0475
0.0475
0.0475
0.0475
0.0475
0.0475
0.0475
0.0475
0.0475
0.0475
0.0475
0.0475
0.0475
0.0475
0.0475
0.0475
0.0475
0.0475
0.0475
0.0475
0.0475
0.0475
0.0475
0.0475
0.0475
0.0475
0.0475
0.0475
0.0475
0.0475
0.0475
0.0475
0.0475
0.0475
0.0475
0.0475
0.0475
0.0475
0.0475
0.0475
0.0475
0.0475
0.0475
0.0475
0.0475
0.0475
0.0475
0.0475
0.0475
0.0475

In [ ]:
filtered_dates = rba_f01d_df[rba_f01d_df['Date'] > (selected_date - pd.DateOffset(years=1))]

filtered_dates

In [145]:

rba_f01d_df

,Date,Cash Rate Target
0,2011-01-05,0.0475
1,2011-01-06,0.0475
2,2011-01-07,0.0475
3,2011-01-10,0.0475
4,2011-01-11,0.0475
...,...,...
3320,2024-02-19,0.0435
3321,2024-02-20,0.0435
3322,2024-02-21,0.0435
3323,2024-02-22,0.0435


# Select what data to run

In [63]:
import os
selected_source = os.listdir('source_yamls')[0]

In [181]:
from pathlib import Path
yaml_path = Path(yamls_folder) / selected_source
yaml_path

WindowsPath('C:/Users/tyewf/github_projects/rba_statistic_tables/source_yamls/Interest Rates and Yields – Money Market – Daily – F1.yaml')

In [169]:
from utils import read_yaml
dataset_yaml = read_yaml('source_yamls/' + selected_source)

## Load data

In [178]:
from data_loading import data_loader

# Get data
file_path, header_descriptions_df, cleaned_df = data_loader(
        dataset_yaml = dataset_yaml,
        data_folder = data_folder,
)

In [ ]:
output_path = Path(outputs_folder) / 'output.xlsx'
cleaned_df.to_excel(output_path, index=False, engine='openpyxl')

In [179]:
file_path
header_descriptions_df
cleaned_df

,Date,Cash Rate Target,Change in the Cash Rate Target,Interbank Overnight Cash Rate,Highest Interbank Overnight Cash Rate,Lowest Interbank Overnight Cash Rate,Volume of Cash Market Transactions,Number of Cash Market Transactions,Total Return Index,EOD 1-month BABs/NCDs,EOD 3-month BABs/NCDs,EOD 6-month BABs/NCDs,1-month OIS,3-month OIS,6-month OIS,1-month Treasury Note,3- month Treasury Note,6- month Treasury Note
0,2011-01-05,0.0475,NaN,0.0475,NaN,NaN,NaN,NaN,1.000130,0.048300,0.049850,0.051400,0.047600,0.047875,0.048500,0.04805,0.04835,NaN
1,2011-01-06,0.0475,NaN,0.0475,NaN,NaN,NaN,NaN,1.000260,0.048300,0.049800,0.051500,0.047600,0.047865,0.048500,0.04800,0.04830,NaN
2,2011-01-07,0.0475,NaN,0.0475,NaN,NaN,NaN,NaN,1.000390,0.048300,0.050100,0.051500,0.047600,0.047900,0.048600,0.04805,0.04830,NaN
3,2011-01-10,0.0475,NaN,0.0475,NaN,NaN,NaN,NaN,1.000781,0.048300,0.050000,0.051450,0.047565,0.047840,0.048465,0.04805,NaN,NaN
4,2011-01-11,0.0475,NaN,0.0475,NaN,NaN,NaN,NaN,1.000911,0.048200,0.049400,0.050750,0.047500,0.047700,0.048000,0.04790,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3317,2024-02-14,0.0435,NaN,0.0432,0.0432,0.0432,913000000.0,4.0,1.316339,0.042950,0.043459,0.044650,NaN,NaN,NaN,NaN,NaN,NaN
3318,2024-02-15,0.0435,NaN,0.0432,NaN,NaN,383000000.0,NaN,1.316495,0.042850,0.043376,0.044542,NaN,NaN,NaN,NaN,NaN,NaN
3319,2024-02-16,0.0435,NaN,0.0432,0.0435,0.0432,954000000.0,9.0,1.316651,0.042950,0.043380,0.044497,NaN,NaN,NaN,NaN,NaN,NaN
3320,2024-02-19,0.0435,NaN,0.0432,0.0432,0.0432,346000000.0,6.0,1.317118,0.042975,0.043400,0.044563,NaN,NaN,NaN,NaN,NaN,NaN


# Select filters

In [764]:
# Arguments
df=df
selected_date = '2023-12-31'
selected_column = default_column
category_column = category_column
selected_category = default_category
aliases_dict = aliases_dict
top_x_category_list = [
  "Rabobank Australia Limited",
  "Australia and New Zealand Banking Group Limited",
  "Sumitomo Mitsui Banking Corporation",
  "The Hongkong and Shanghai Banking Corporation Limited",
  "Suncorp-Metway Limited",
  "Mizuho Bank, Ltd.",
  "Bendigo and Adelaide Bank Limited",
  "Macquarie Bank Limited",
  "National Australia Bank Limited",
  "Commonwealth Bank of Australia",
  "Bank of China Limited",
  "Bank of America, National Association",
  "MUFG Bank, Ltd.",
  "Westpac Banking Corporation",
  "ING Bank (Australia) Limited",
  "Bank of Queensland Limited"
]

# Filter Data

In [806]:
from data_filtering import filter_data

dfs_dict, details_dicts = filter_data(
    df = df,
    date_column = date_column,
    selected_date = selected_date,
    selected_column = selected_column,
    category_column = category_column,
    selected_category = selected_category,
    top_x_category_list = top_x_category_list,
    group_by_columns = group_by_columns,
)

# print(details_dicts.keys())
# print(dfs_dict.keys())

c:\Users\tyewf\github_projects\rba_monthly_adi_statistics_excel\data_filtering.py:286: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

c:\Users\tyewf\github_projects\rba_monthly_adi_statistics_excel\data_filtering.py:286: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

c:\Users\tyewf\github_projects\rba_monthly_adi_statistics_excel\data_filtering.py:286: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the c

In [814]:
dfs_dict['month']['df']

Period,Institution Name,comparison,current,Business Loans - Month Movement ($),Business Loans - Month Movement Direction,Business Loans - Month Movement (%),Business Loans - Month Movement as Percentage of Market(%)
0,Australia and New Zealand Banking Group Limited,1.547812e+11,1.559335e+11,1.152300e+09,increase,0.007445,0.000980
1,"Bank of America, National Association",2.005810e+10,1.940420e+10,-6.539000e+08,decrease,-0.032600,-0.000556
2,Bank of China Limited,1.910160e+10,2.045380e+10,1.352200e+09,increase,0.070790,0.001150
3,Bank of Queensland Limited,1.394710e+10,1.383450e+10,-1.126000e+08,decrease,-0.008073,-0.000096
4,Bendigo and Adelaide Bank Limited,1.622990e+10,1.622330e+10,-6.600000e+06,decrease,-0.000407,-0.000006
5,Commonwealth Bank of Australia,2.032435e+11,2.040256e+11,7.821000e+08,increase,0.003848,0.000665
6,ING Bank (Australia) Limited,1.375580e+10,1.369890e+10,-5.690000e+07,decrease,-0.004136,-0.000048
7,"MUFG Bank, Ltd.",2.246010e+10,2.227300e+10,-1.871000e+08,decrease,-0.008330,-0.000159
8,Macquarie Bank Limited,1.909380e+10,1.980800e+10,7.142000e+08,increase,0.037405,0.000608
9,"Mizuho Bank, Ltd.",1.966650e+10,1.930120e+10,-3.653000e+08,decrease,-0.018575,-0.000311


In [744]:
# Sample DataFrame
df = pd.DataFrame({
    'date': ['2021-01-01', '2021-01-01', '2021-01-02', '2021-01-02'],
    'variable': ['A', 'B', 'A', 'B'],
    'value': [1, 2, 3, 4]
})

# Pivot the DataFrame
pivoted_df = df.pivot(index='date', columns='variable', values='value')

print(pivoted_df)

variable    A  B
date            
2021-01-01  1  2
2021-01-02  3  4


In [734]:
dfs_dict['market_position_df']
# dfs_dict['top_x_df_dict']['df'][[date_column, category_column, selected_column]]

details_dicts['months_ago_list']

[1, 12, 48]

## Descriptions

In [736]:
from descriptions import generate_descriptions

descriptions_dict = generate_descriptions(
    dfs_dict = dfs_dict,
    date_column = date_column,
    selected_column = selected_column,
    category_column = category_column,
    selected_category = selected_category,
    aliases_dict = aliases_dict,
    details_dicts = details_dicts,
)

# Chart

In [22]:
close_log_handlers()

In [709]:
from charting.utils_charting import generate_charts

charts_dict = generate_charts(
    dfs_dict = dfs_dict,
    details_dicts = details_dicts,
    date_column = date_column,
    selected_date = selected_date,
    category_column = category_column,
    selected_category = selected_category,
    selected_column = selected_column,
    top_x_category_list = top_x_category_list,
    color_discrete_map = color_discrete_map,
)


c:\Users\tyewf\github_projects\rba_monthly_adi_statistics_excel\chart_generator.py:129: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\tyewf\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\plotly\express\_core.py:2065: FutureWarning:

When grouping with a length-1 list-like, you will need to pass a length-1 tuple to get_group in a future version of pandas. Pass `(name,)` instead of `name` to silence this warning.



In [710]:
for key in charts_dict.keys():
    print(key)
#charts_dict['fig_balance_as_at'].show()
charts_dict['Business Loans']


Business Loans


In [711]:
from charting.utils_charting import chart_selected_col_bar
from utils import get_months_ago_list

top_x_df = dfs_dict['top_x_df_dict']['df']
top_x_df
details_dicts

# Get number of months within data
months_ago_list = get_months_ago_list(
    df = top_x_df,
    date_column = date_column
)
months_ago_list


## 'date_0_months_ago': Timestamp('2023-12-31 00:00:00'),
##  'date_0_wording': 'current month',
##  'date_1_months_ago': Timestamp('2023-11-30 00:00:00'),
##  'date_1_wording': 'month',
##  'date_12_months_ago': Timestamp('2022-12-31 00:00:00'),
##  'date_12_wording': 'year',
##  'date_48_months_ago': Timestamp('2019-12-31 00:00:00'),
##  'date_48_wording': '4 years',

# top_x_df_current = top_x_df[top_x_df[date_column] == selected_date]
# formatted_dollars  = top_x_df_current[selected_column].apply(rounded_dollars)
# formatted_percentages = top_x_df_current['Market Share'].apply(lambda x: f"({x * 100:.1f} %)")
# top_x_df_current.loc[:, 'chart_txt'] = formatted_dollars + ' ' + formatted_percentages

# charts_dict[selected_column] = chart_selected_col_bar(
#         df = top_x_df_current,
#         category_column = category_column,
#         reference_col = f'{selected_column}',
#         ordered_category_list= details_dicts['ordered_category_list'],
#         show_xaxis_labels = True,
#         x_tickformat = None,
#         x_gridcolor='Grey',
#         color_discrete_map= color_discrete_map,
#     )

[1, 12, 48]

In [436]:
import plotly.express as px
fig = px.bar(
    data_frame=top_x_df_at_date,
    x=top_x_df_at_date['Market Share'] * 100,
    y=top_x_df_at_date[category_column],
    orientation='h',
    title=f"Test",
    color=category_column,
    # hover_name=None, hover_data=None, custom_data=None,
    text=top_x_df_at_date['txt'],
    # base=None, error_x=None, error_x_minus=None, error_y=None,
    # error_y_minus=None, animation_frame=None, animation_group=None,
    category_orders={category_column: ordered_category_list},  # Ensure custom order is applied
    # labels=None,
    color_discrete_sequence=[color_discrete_map['default_color']],
    color_discrete_map=color_discrete_map, # color_continuous_scale=None, pattern_shape_sequence=None, pattern_shape_map=None, range_color=None, color_continuous_midpoint=None, opacity=None,
    
    # barmode='relative', log_x=False, log_y=False, range_x=None, range_y=None, text_auto=False, title=None, template=None, width=None, height=None
)

# orientation='h',
# title=f"{reference_col}",
fig.update_layout(
    text=top_x_df_at_date['txt'])

fig.upda

# height=800

C:\Users\tyewf\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\plotly\express\_core.py:2065: FutureWarning:

When grouping with a length-1 list-like, you will need to pass a length-1 tuple to get_group in a future version of pandas. Pass `(name,)` instead of `name` to silence this warning.



ValueError: Invalid property specified for object of type plotly.graph_objs.Layout: 'text'

Did you mean "font"?

    Valid properties:
        activeselection
            :class:`plotly.graph_objects.layout.Activeselection`
            instance or dict with compatible properties
        activeshape
            :class:`plotly.graph_objects.layout.Activeshape`
            instance or dict with compatible properties
        annotations
            A tuple of
            :class:`plotly.graph_objects.layout.Annotation`
            instances or dicts with compatible properties
        annotationdefaults
            When used in a template (as
            layout.template.layout.annotationdefaults), sets the
            default property values to use for elements of
            layout.annotations
        autosize
            Determines whether or not a layout width or height that
            has been left undefined by the user is initialized on
            each relayout. Note that, regardless

In [ ]:
go.Bar(
        name=segment,
        x=[share],  # Each segment's share
        y=['Market Share'],  # Common 'category' for all segments
        orientation='h',  # Make the bar horizontal        
        text=f"{share}%",  # Optionally display the percentage on the segment
        textposition='outside',
    ))

In [380]:
import plotly.graph_objects as go

# Market share segments and values
## segments = dfs_dict['market_position_df']['Institution Name']
## shares = dfs_dict['market_position_df']['Market Share'] * 100

fig = go.Figure()

# Accumulator to keep track of the x position for the annotation
accumulated_share = 0

# Segment to annotate
segment_to_annotate = 'Segment B'
annotation_text = ''  # We'll fill this in with the company's position and amount

for segment, share in zip(segments, shares):
    fig.add_trace(go.Bar(
        name=segment,
        x=[share],  # Each segment's share
        y=['Market Share'],  # Common 'category' for all segments
        orientation='h',  # Make the bar horizontal        
        text=f"{share}%",  # Optionally display the percentage on the segment
        textposition='outside',
    ))

    
    if segment == segment_to_annotate:
        # Set up annotation text
        annotation_text = f"{segment}: {share}%"
        # Position the annotation at the end of the current segment
        position_for_annotation = accumulated_share + share / 2
    
    # Update the accumulated share
    accumulated_share += share

# Add the annotation to the figure
fig.add_annotation(
    x=position_for_annotation,
    y='Market Share',
    text=annotation_text,
    showarrow=True,
    arrowhead=1,
    ax=20,  # Adjust arrow length and direction
    ay=-30,
)

# Update layout with stack mode and adjust other settings for aesthetics
fig.update_layout(
    title='Market Share by Segment',
    xaxis=dict(
        title='Percentage',
        ticksuffix="%",  # Add a percentage sign to x-axis ticks
        range=[0, 100]  # Ensure x-axis goes from 0 to 100 for the percentages
    ),
    yaxis=dict(
        title='',
        showticklabels=False  # Hide y-axis tick labels since there's only one 'category'
    ),
    barmode='stack',
    uniformtext_minsize=8, 
    uniformtext_mode='hide',
    legend_title_text='Segment',
    #title=f"Market Share",
    #color=category_column,
    category_orders={category_column: ordered_category_list},  # Ensure custom order is applied
    #color_discrete_map=color_discrete_map,
    #color_discrete_sequence=[color_discrete_map['default_color']],
)

fig.show()


ValueError: Invalid property specified for object of type plotly.graph_objs.Layout: 'category'

Did you mean "barnorm"?

    Valid properties:
        activeselection
            :class:`plotly.graph_objects.layout.Activeselection`
            instance or dict with compatible properties
        activeshape
            :class:`plotly.graph_objects.layout.Activeshape`
            instance or dict with compatible properties
        annotations
            A tuple of
            :class:`plotly.graph_objects.layout.Annotation`
            instances or dicts with compatible properties
        annotationdefaults
            When used in a template (as
            layout.template.layout.annotationdefaults), sets the
            default property values to use for elements of
            layout.annotations
        autosize
            Determines whether or not a layout width or height that
            has been left undefined by the user is initialized on
            each relayout. Note that, reg

In [248]:
import plotly.graph_objects as go

# Market share segments (assuming these are parts of a whole for a single category)
segments = ['Segment A', 'Segment B', 'Segment C']
shares = [40, 35, 25]  # Represents the market shares in percentages

fig = go.Figure()

# Add a trace for each segment of the market share
for segment, share in zip(segments, shares):
    fig.add_trace(go.Bar(
        name=segment,
        x=[share],  # Each segment's share
        y=['Market Share'],  # Common 'category' for all segments
        orientation='h',  # Make the bar horizontal
        text=f"{share}%",  # Display the percentage on the segment
        textposition='inside',
    ))

# Update the layout for a 100% stacked look
fig.update_layout(
    title='Market Share by Segment',
    xaxis=dict(
        title='Percentage',
        ticksuffix="%",  # Add a percentage sign to x-axis ticks
        range=[0, 100]  # Ensure x-axis goes from 0 to 100 for the percentages
    ),
    yaxis=dict(
        title='',  # No specific title needed for a single category
        showticklabels=False  # Hide y-axis tick labels since there's only one 'category'
    ),
    barmode='stack',
    uniformtext_minsize=8, 
    uniformtext_mode='hide',
    legend_title_text='Segment'
)

fig.show()


In [245]:
import plotly.graph_objects as go

aggregates_df = dfs_dict['aggregates_df_dict']['df']
aggregates_df_category_col = dfs_dict['aggregates_df_dict']['category_col']
aggregates_df_dollar_col = dfs_dict['aggregates_df_dict']['dollar_col']
aggregates_df_dollar_movements_col = dfs_dict['aggregates_df_dict']['dollar_movements_col']
aggregates_df_percentage_movements_col = dfs_dict['aggregates_df_dict']['percentage_movements_col']

aggregates_df_selected_date = aggregates_df[aggregates_df[date_column] == selected_date]
aggregates_df_selected_date.head(5)

categories = aggregates_df_selected_date[aggregates_df_category_col]
values = aggregates_df_selected_date[aggregates_df_dollar_col]

# Create the treemap
fig = go.Figure(go.Treemap(
    labels=categories,
    values=values,
    parents=[""] * len(categories),  # Indicates no parent for all items
    textinfo="label+value"  # Display labels and values on the treemap
))

fig.update_layout(margin=dict(t=50, l=25, r=25, b=25))

fig.show()


In [ ]:
import plotly.graph_objects as go

fig = go.Figure(go.Treemap(
    labels=["Parent", "Child 1", "Child 2", "Child 3"],  # Names of each sector
    parents=["", "Parent", "Parent", "Parent"],  # Defines the hierarchy
    values=[10, 20, 15, 5],  # Size of each sector
    textinfo="label+value"
))

fig.update_layout(margin=dict(t=50, l=25, r=25, b=25))

fig.show()


In [48]:
from charting.utils_charting import growth_over_time

fig = growth_over_time(
    category_df = dfs_dict['top_x_df'][dfs_dict['top_x_df'][category_column] == selected_category].copy(),
    date_column = date_column,
    chart_column = f'{selected_column} Movement ($)',
)
fig


In [46]:
dfs_dict['top_x_df']['Business Loans Movement ($)']

,Period,Institution Name,Business Loans,Business Loans Movement ($),Business Loans Movement (%)
0,2019-03-31,Australia and New Zealand Banking Group Limited,1.305555e+11,NaN,NaN
17,2019-04-30,Australia and New Zealand Banking Group Limited,1.265449e+11,-4.010600e+09,-0.030720
34,2019-05-31,Australia and New Zealand Banking Group Limited,1.266298e+11,8.490000e+07,0.000671
51,2019-06-30,Australia and New Zealand Banking Group Limited,1.270493e+11,4.195000e+08,0.003313
68,2019-07-31,Australia and New Zealand Banking Group Limited,1.293160e+11,2.266700e+09,0.017841
...,...,...,...,...,...
917,2023-08-31,other,1.612986e+11,4.277000e+09,0.027238
934,2023-09-30,other,1.624169e+11,1.118300e+09,0.006933
951,2023-10-31,other,1.621575e+11,-2.594000e+08,-0.001597
968,2023-11-30,other,1.630558e+11,8.983000e+08,0.005540


In [45]:
import pandas as pd
import plotly.graph_objects as go
import datetime as dt

category_df = dfs_dict['top_x_df'][dfs_dict['top_x_df'][category_column] == selected_category].copy()
category_df

# Preparing the DataFrame
category_df['Year'] = category_df[date_column].dt.year
category_df['Month'] = category_df[date_column].dt.strftime('%B')  # Full month name
category_df['MonthIndex'] = category_df[date_column].dt.month  # To ensure correct month sorting

pivot_df = category_df.pivot_table(index=['Month', 'MonthIndex'], columns='Year', values=selected_column, aggfunc='sum').reset_index().sort_values('MonthIndex')
pivot_df

# Plotly graph
fig = go.Figure()

# Add a bar trace for each year
for year in pivot_df.columns[2:]:  # Skipping 'Month' and 'MonthIndex' columns
    fig.add_trace(go.Bar(
        x=pivot_df['Month'],
        y=pivot_df[year],
        name=str(year),
    ))

# Customize layout
fig.update_layout(
    title='Monthly Values Over Years',
    xaxis_title='Month',
    yaxis_title='Value',
    barmode='group',  # Grouped bar chart
    xaxis={'categoryorder':'array', 'categoryarray':pivot_df['Month']}  # Ensure correct month order
)

# Show figure
fig.show()

In [ ]:
import pandas as pd
import plotly.graph_objects as go

# Sample data creation
# Create a date range from January of the first year to December of the fifth year
dates = pd.date_range(start='2016-01-01', end='2020-12-31', freq='M')

# Create sample data for demonstration
import numpy as np
data = {
    'Date': dates,
    'Series1': np.random.rand(len(dates)) * 1000,  # Example data series
    'Series2': np.random.rand(len(dates)) * 1000,  # Another example data series
    # Add more series if needed
}

df = pd.DataFrame(data)

# Convert 'Date' to datetime and extract year and month for grouping
df['Year'] = df['Date'].dt.year
df['Month'] = df['Date'].dt.strftime('%b')  # Convert month to abbreviated name

# Plotly graph
fig = go.Figure()

# Add traces for each year
for year in df['Year'].unique():
    for series in ['Series1', 'Series2']:  # Loop through each data series
        fig.add_trace(go.Bar(
            x=df[df['Year'] == year]['Month'],
            y=df[df['Year'] == year][series],
            name=f'{series} {year}',
            # Customize further as needed
        ))

# Customize layout
fig.update_layout(
    title='Monthly Data for 5 Years',
    xaxis_title='Month',
    yaxis_title='Value',
    barmode='group',  # Use 'group' for grouped bar chart, 'stack' for stacked
)

# Show figure
fig.show()


In [21]:
charts_dict[selected_column]
dfs_dict['original_df']

In [ ]:
import plotly.graph_objects as go

def chart_treemap(
        categories,
        values,
):

    # Create the treemap
    fig = go.Figure(go.Treemap(
        labels=categories,
        values=values,
        parents=[""] * len(categories),  # Indicates no parent for all items
        textinfo="label+value"  # Display labels and values on the treemap
    ))

    fig.update_layout(margin=dict(t=50, l=25, r=25, b=25))

    fig.show()

aggregates_df = dfs_dict['aggregates_df']
aggregates_df_dated = aggregates_df[aggregates_df[date_column] == selected_date]


chart_treemap(
        categories = aggregates_df_dated['Account'],
        values = aggregates_df_dated['Value'],
)

# Tabs

In [33]:
from utils import rounded_dollars_md, escape_dollar_signs

details_dicts['ordered_category_list']
aggregates_df = dfs_dict['aggregates_df']
aggregates_df_account = aggregates_df[aggregates_df['Account'] == selected_column]


rounded_dollars_md(aggregates_df_account['Value'].iloc[-1])
aggregates_df_account['Value'].iloc[-2]
escape_dollar_signs(rounded_dollars_md(aggregates_df_account['Value'].iloc[-1]))

INFO:utils:1.18
INFO:utils:1.18


'\\$&nbsp;1.18&nbsp;Trn'

In [73]:
aggregates_df_account
dfs_dict['aggregates_df']
# dfs_dict['top_x_df']

,Period,Account,Value,Value Movement ($),Value Movement (%)
928,2019-03-31,Acceptances,2.654000e+08,NaN,NaN
929,2019-04-30,Acceptances,3.185000e+08,5.310000e+07,0.200075
930,2019-05-31,Acceptances,3.752000e+08,5.670000e+07,0.178022
931,2019-06-30,Acceptances,6.384000e+08,2.632000e+08,0.701493
932,2019-07-31,Acceptances,2.715000e+08,-3.669000e+08,-0.574718
...,...,...,...,...,...
111,2023-08-31,Trading securities,8.481230e+10,-2.611500e+09,-0.029872
112,2023-09-30,Trading securities,8.460500e+10,-2.073000e+08,-0.002444
113,2023-10-31,Trading securities,8.424410e+10,-3.609000e+08,-0.004266
114,2023-11-30,Trading securities,9.157700e+10,7.332900e+09,0.087043


In [53]:
from IPython.display import display, Markdown, Latex
display(Markdown(escape_dollar_signs(rounded_dollars_md(aggregates_df_account['Value'].iloc[-1]))))
tst = Markdown(escape_dollar_signs(rounded_dollars_md(aggregates_df_account['Value'].iloc[-1])))
print(tst)
display(Markdown('**Hi** hello'))

INFO:utils:1.18


\$&nbsp;1.18&nbsp;Trn

INFO:utils:1.18


<IPython.core.display.Markdown object>


**Hi** hello

In [49]:
escape_dollar_signs(
        f" - The Net Total for {reference_col} is {rounded_dollars_md(df[reference_col].sum())}"
    )

NameError: name 'reference_col' is not defined